# Question 1

## 1. Import libraries

In [0]:
import pandas as pd
import numpy as np
import codecs
from bs4 import BeautifulSoup

## 2. Download and read data to a data frame

In [196]:
!wget https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print("Data downloaded successfully!")

--2020-01-15 09:13:56--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Resolving en.wikipedia.org (en.wikipedia.org)... 208.80.154.224, 2620:0:863:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.154.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78806 (77K) [text/html]
Saving to: ‘List_of_postal_codes_of_Canada:_M’

List_of_postal_code 100%[===================>]  76.96K   489KB/s    in 0.2s    

2020-01-15 09:13:57 (489 KB/s) - ‘List_of_postal_codes_of_Canada:_M’ saved [78806/78806]

Data downloaded successfully!


In [0]:
file = codecs.open('List_of_postal_codes_of_Canada:_M', encoding='utf-8')
html_doc = file.read()
soup = BeautifulSoup(html_doc, 'html.parser')
df = (pd.read_html(str(soup.table))[0])

Check original shape of the data frame

In [198]:
df.shape

(287, 3)

See a few first rows of the data frame

In [199]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## 3. Remove rows with 'Not assigned' borough

First, check all unique values of the Borough column to see 'Not assigned' variations

In [200]:
# Check unique values of the Borough columns
df['Borough'].unique()

array(['Not assigned', 'North York', 'Downtown Toronto', "Queen's Park",
       'Scarborough', 'East York', 'Etobicoke', 'York', 'East Toronto',
       'West Toronto', 'Central Toronto', 'Mississauga'], dtype=object)

We can see that there is only one not-assigned value with the format 'Not assgined'. </br>
Second, count all 'Not assigned' borough

In [201]:
df[df['Borough'] == 'Not assigned']

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
8,M8A,Not assigned,Not assigned
12,M2B,Not assigned,Not assigned
19,M7B,Not assigned,Not assigned
...,...,...,...
277,M4Z,Not assigned,Not assigned
278,M5Z,Not assigned,Not assigned
279,M6Z,Not assigned,Not assigned
280,M7Z,Not assigned,Not assigned


There are 77 rows with 'Not assigned' values. After the remove, the data frame should have 210 rows.

In [202]:
df = df[df['Borough'] != "Not assigned"].reset_index(drop=True)
df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West
206,M8Z,Etobicoke,Mimico NW
207,M8Z,Etobicoke,The Queensway West
208,M8Z,Etobicoke,Royal York South West


Finally, check the unique values of borough column once again.

In [203]:
df['Borough'].unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Scarborough',
       'East York', 'Etobicoke', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

## 4. Update 'not assigned' neighborhoods with borough names

Check all unique values of Neighbourhood column

In [204]:
df['Neighbourhood'].unique()

array(['Parkwoods', 'Victoria Village', 'Harbourfront',
       'Lawrence Heights', 'Lawrence Manor', "Queen's Park",
       'Not assigned', 'Rouge', 'Malvern', 'Don Mills North',
       'Woodbine Gardens', 'Parkview Hill', 'Ryerson', 'Garden District',
       'Glencairn', 'Cloverdale', 'Islington', 'Martin Grove',
       'Princess Gardens', 'West Deane Park', 'Highland Creek',
       'Rouge Hill', 'Port Union', 'Flemingdon Park', 'Don Mills South',
       'Woodbine Heights', 'St. James Town', 'Humewood-Cedarvale',
       'Bloordale Gardens', 'Eringate', 'Markland Wood',
       'Old Burnhamthorpe', 'Guildwood', 'Morningside', 'West Hill',
       'The Beaches', 'Berczy Park', 'Caledonia-Fairbanks', 'Woburn',
       'Leaside', 'Central Bay Street', 'Christie', 'Cedarbrae',
       'Hillcrest Village', 'Bathurst Manor', 'Downsview North',
       'Wilson Heights', 'Thorncliffe Park', 'Adelaide', 'King',
       'Richmond', 'Dovercourt Village', 'Dufferin',
       'Scarborough Village', 'Fairv

Check the number of neighborhoods with 'Not assigned' name

In [205]:
not_assigned = df[df['Neighbourhood']=='Not assigned']
not_assigned

,Postcode,Borough,Neighbourhood
6,M9A,Queen's Park,Not assigned


Update neighbourhood name with borough name

In [0]:
for i in range(len(not_assigned)):
    row_index = not_assigned.index[0]
    df.at[row_index, 'Neighbourhood'] = df.at[row_index, 'Borough']

Check neighborhood name after update

In [207]:
df[df['Neighbourhood']=='Not assigned']

,Postcode,Borough,Neighbourhood


In [208]:
df[6:7]

,Postcode,Borough,Neighbourhood
6,M9A,Queen's Park,Queen's Park


## 5. Group all neighbourhoods that have the same postal code

Group neighbourhoods by postal codes

In [209]:
aggregations = {
    'Neighbourhood':lambda x:', '.join(x)}
grouped_df = df.groupby('Postcode').agg(aggregations).reset_index()
grouped_df

,Postcode,Neighbourhood
0,M1B,"Rouge, Malvern"
1,M1C,"Highland Creek, Rouge Hill, Port Union"
2,M1E,"Guildwood, Morningside, West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae
...,...,...
98,M9N,Weston
99,M9P,Westmount
100,M9R,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,"Albion Gardens, Beaumond Heights, Humbergate, ..."


Add borough column corresponding to postal code

In [210]:
# Define a data frame with postcode and borough
pcode_borough_df = df.drop('Neighbourhood',axis=1)
pcode_borough_df

,Postcode,Borough
0,M3A,North York
1,M4A,North York
2,M5A,Downtown Toronto
3,M6A,North York
4,M6A,North York
...,...,...
205,M8Z,Etobicoke
206,M8Z,Etobicoke
207,M8Z,Etobicoke
208,M8Z,Etobicoke


In [211]:
# Merge 2 data frames
merged_df = pd.merge(pcode_borough_df, grouped_df, on='Postcode', how='left')
merged_df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
...,...,...,...
205,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."
206,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."
207,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."
208,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."


In [212]:
# Remove duplicates
clean_df = merged_df.drop_duplicates()
clean_df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M7A,Downtown Toronto,Queen's Park
...,...,...,...
192,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
195,M4Y,Downtown Toronto,Church and Wellesley
196,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
197,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."


In [0]:
# Save the clean data frame to csv file
clean_df.to_csv("toronto_data_1.csv", index=False)

In [214]:
# Print shape of the clean data frame
clean_df.shape

(103, 3)